In [1]:
import meep as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

sys.path.append('../../..')
from meep_metamaterials.lithography import lithography as lt

Using MPI version 4.0, 1 processes


/home/gridsan/mcomella/.conda/envs/pmp/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gridsan/mcomella/.conda/envs/pmp/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/gridsan/mcomella/.conda/envs/pmp/lib/python3.11/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/gridsan/mcomella/.conda/envs/pmp/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


We used 285 nm pixels with a dwell time of 2 us, which is a speed of 142 mm/s.

In [2]:
powers = np.linspace(0.1, 1, 8) # Range of powers
speeds = np.linspace(50, 160, 8) # Range of speeds in mm/s

pixel_size = 285e-6 # 500 nm (size in mm)
dwell_time = 2e-6 # Dwell time in s
dz = 1e-3 # 1 um

T = 0.8 # Period in mm
w = 0.4 # Width in mm
t = 5e-3 # Thickness in mm

gel_size = 10 # Size of gel in mm

x_aspect = 0.1

In [3]:
# Scale to match the printer window width
w = w/x_aspect
pixel_size = pixel_size/x_aspect

In [4]:
block = mp.Block(mp.Vector3(w,w,t))

n, m = len(powers), len(speeds)
i, j = 0, 0
mat = lt.draw_block(block, 0, 0, speed=1, power=1, dx=pixel_size, dz=dz)

100%|██████████| 6/6 [00:23<00:00,  3.84s/it]


In [5]:
M = pd.DataFrame(columns=['xi', 'yi', 'pi', 'xf', 'yf', 'pf', 't', 'X', 'Y', 'Z'])

# Pmax = 0.3 # Maximum power in W

n, m = len(powers), len(speeds)
i, j = 0, 0
for pwr in powers:
    for speed in speeds:
        mt = mat.copy()

        mt['t'] = w*x_aspect/speed
        mt['pi'] = pwr
        mt['pf'] = pwr
        x_offset = (i-n/2+1/2) * T
        y_offset = (j-m/2+1/2) * T

        mt['X'] = mt['X'] + x_offset
        mt['Y'] = mt['Y'] + y_offset

        j += 1
        M = pd.concat([M, mt])
        
    j = 0
    i += 1

In [6]:
# Total time
print(M['t'].sum()/3600, 'h')
print(M['t'].sum()/60, 'min')

0.6560344705565528 h
39.362068233393174 min


In [46]:
lt.save_to_matlab(M, '../instructions/dielectric_properties/substrate.mat')

---

Add lines on top

In [7]:
sep = 1.2e-3 # Separation between lines in grating
dc = 0.5 # Duty cycle
t = 3e-3 # Thickness of grating

geometry = [mp.Block(size=mp.Vector3(10*sep*dc, 10*sep, t))]

mat = lt.draw_metamaterial(geometry, a=10*sep, nrows=30, ncols=30, dx=pixel_size, dz=dz, speed=speed)

100%|██████████| 30/30 [00:01<00:00, 16.62it/s]


In [48]:
M = pd.DataFrame(columns=['xi', 'yi', 'pi', 'xf', 'yf', 'pf', 't', 'X', 'Y', 'Z'])

n, m = len(powers), len(speeds)
i, j = 0, 0
for pwr in powers:
    for speed in speeds:
        mt = mat.copy()

        mt['t'] = w*x_aspect/speed
        mt['pi'] = pwr
        mt['pf'] = pwr
        x_offset = (i-n/2+1/2) * T
        y_offset = (j-m/2+1/2) * T

        mt['X'] = mt['X'] + x_offset
        mt['Y'] = mt['Y'] + y_offset

        mt['Z'] = mt['Z'] + t/2

        j += 1
        M = pd.concat([M, mt])
        
    j = 0
    i += 1

In [49]:
# Total time
print(M['t'].sum()/3600, 'h')
print(M['t'].sum()/60, 'min')

0.8410698340468629 h
50.46419004281177 min


In [50]:
lt.save_to_matlab(M, '../instructions/dielectric_properties/grating.mat')

---

In [51]:
m1 = lt.load_from_matlab('../instructions/dielectric_properties/substrate.mat')
m2 = lt.load_from_matlab('../instructions/dielectric_properties/grating.mat')

lt.save_to_matlab(pd.concat([m1, m2]), '../instructions/dielectric_properties/substrate_grating.mat')